In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

In [2]:
# Training settings
parser = argparse.ArgumentParser()
# 禁用CUDA训练
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
# 在训练通过期间验证
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
# 随机种子
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
# 要训练的epoch数
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
# 最初的学习率
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
# 权重衰减（参数L2损失）
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
# 隐藏层单元数量
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
# dropout率（1-保持概率)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

# args = parser.parse_args()
args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

# 产生随机种子，以使得结果是确定的
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [3]:
# Load data
# 加载数据
# adj: adj样本关系的对称邻接矩阵的稀疏张量
# features: 样本特征张量
# labels: 样本标签
# idx_train: 训练集索引列表
# idx_val: 验证集索引列表
# idx_test: 测试集索引列表
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [4]:
adj

tensor(indices=tensor([[   0,    8,   14,  ..., 1389, 2344, 2707],
                       [   0,    0,    0,  ..., 2707, 2707, 2707]]),
       values=tensor([0.1667, 0.1667, 0.0500,  ..., 0.2000, 0.5000, 0.2500]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)

In [5]:
features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
labels

tensor([0, 4, 1,  ..., 6, 5, 0])

In [7]:
idx_train

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [8]:
idx_val

tensor([200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,
        228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241,
        242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
        256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269,
        270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283,
        284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297,
        298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
        312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325,
        326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339,
        340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353,
        354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
        368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 3

In [9]:
idx_test

tensor([ 500,  501,  502,  503,  504,  505,  506,  507,  508,  509,  510,  511,
         512,  513,  514,  515,  516,  517,  518,  519,  520,  521,  522,  523,
         524,  525,  526,  527,  528,  529,  530,  531,  532,  533,  534,  535,
         536,  537,  538,  539,  540,  541,  542,  543,  544,  545,  546,  547,
         548,  549,  550,  551,  552,  553,  554,  555,  556,  557,  558,  559,
         560,  561,  562,  563,  564,  565,  566,  567,  568,  569,  570,  571,
         572,  573,  574,  575,  576,  577,  578,  579,  580,  581,  582,  583,
         584,  585,  586,  587,  588,  589,  590,  591,  592,  593,  594,  595,
         596,  597,  598,  599,  600,  601,  602,  603,  604,  605,  606,  607,
         608,  609,  610,  611,  612,  613,  614,  615,  616,  617,  618,  619,
         620,  621,  622,  623,  624,  625,  626,  627,  628,  629,  630,  631,
         632,  633,  634,  635,  636,  637,  638,  639,  640,  641,  642,  643,
         644,  645,  646,  647,  648,  6

In [5]:
# Model and optimizer
# 模型和优化器

# GCN模型
# nfeat输入单元数，shape[1]表示特征矩阵的维度数（列数）
# nhid中间层单元数量
# nclass输出单元数，即样本标签数=样本标签最大值+1
# dropout参数
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)

# 构造一个优化器对象Optimizer，用来保存当前的状态，并能够根据计算得到的梯度来更新参数
# Adam优化器
# lr学习率
# weight_decay权重衰减（L2惩罚）
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

In [6]:
# 如果使用GPU则执行这里，数据写入cuda，便于后续加速
if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [7]:
# 定义训练函数
def train(epoch):
    # 返回当前时间
    t = time.time()
    
    # train的时候使用dropout, 测试的时候不使用dropout
    # pytorch里面eval()固定整个网络参数，没有dropout
    
    # 固定语句，主要针对启用BatchNormalization和Dropout
    model.train()
    
    # 把梯度置零，也就是把loss关于weight的导数变成0
    optimizer.zero_grad()
    # 执行GCN中的forward前向传播
    output = model(features, adj)
    # 最大似然/log似然损失函数，idx_train是140(0~139)
    # nll_loss: negative log likelihood loss
    # https://www.cnblogs.com/marsggbo/p/10401215.html
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    # 准确率
    acc_train = accuracy(output[idx_train], labels[idx_train])
    # 反向传播
    loss_train.backward()
    # 梯度下降，更新值
    optimizer.step()

    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    # 是否在训练期间进行验证
    if not args.fastmode:
        # 固定语句，主要针对不启用BatchNormalization和Dropout
        model.eval()
        # 前向传播
        output = model(features, adj)
    
    # 最大似然/log似然损失函数，idx_val是300(200~499)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    # 准确率
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    # 正在迭代的epoch数
    # 训练集损失函数值
    # 训练集准确率
    # 验证集损失函数值
    # 验证集准确率
    # 运行时间
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [8]:
# 定义测试函数，相当于对已有的模型在测试集上运行对应的loss与accuracy
def test():
    # 固定语句，主要针对不启用BatchNormalization和Dropout
    model.eval()
    # 前向传播
    output = model(features, adj)
    # 最大似然/log似然损失函数，idx_test是1000(500~1499)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    # 准确率
    acc_test = accuracy(output[idx_test], labels[idx_test])
    
    # 测试集损失函数值
    # 测试集的准确率
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [9]:
# Train model
t_total = time.time()
# epoch数
for epoch in range(args.epochs):
    # 训练
    train(epoch)
print("Optimization Finished!")
# 已用总时间
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.9390 acc_train: 0.2000 loss_val: 1.9045 acc_val: 0.3500 time: 4.4010s
Epoch: 0002 loss_train: 1.9161 acc_train: 0.2929 loss_val: 1.8941 acc_val: 0.3500 time: 0.0080s
Epoch: 0003 loss_train: 1.9125 acc_train: 0.2929 loss_val: 1.8844 acc_val: 0.3500 time: 0.0070s
Epoch: 0004 loss_train: 1.9027 acc_train: 0.2929 loss_val: 1.8751 acc_val: 0.3500 time: 0.0105s
Epoch: 0005 loss_train: 1.8933 acc_train: 0.2929 loss_val: 1.8660 acc_val: 0.3500 time: 0.0076s
Epoch: 0006 loss_train: 1.8759 acc_train: 0.2929 loss_val: 1.8574 acc_val: 0.3500 time: 0.0079s
Epoch: 0007 loss_train: 1.8626 acc_train: 0.2929 loss_val: 1.8490 acc_val: 0.3500 time: 0.0088s
Epoch: 0008 loss_train: 1.8679 acc_train: 0.2929 loss_val: 1.8410 acc_val: 0.3500 time: 0.0082s
Epoch: 0009 loss_train: 1.8558 acc_train: 0.2929 loss_val: 1.8335 acc_val: 0.3500 time: 0.0073s
Epoch: 0010 loss_train: 1.8452 acc_train: 0.2929 loss_val: 1.8262 acc_val: 0.3500 time: 0.0089s
Epoch: 0011 loss_train: 1.8428 acc_train

Epoch: 0092 loss_train: 0.8695 acc_train: 0.8214 loss_val: 1.0585 acc_val: 0.7700 time: 0.0110s
Epoch: 0093 loss_train: 0.8396 acc_train: 0.8357 loss_val: 1.0505 acc_val: 0.7800 time: 0.0080s
Epoch: 0094 loss_train: 0.8665 acc_train: 0.8429 loss_val: 1.0421 acc_val: 0.7800 time: 0.0075s
Epoch: 0095 loss_train: 0.8840 acc_train: 0.8143 loss_val: 1.0345 acc_val: 0.7833 time: 0.0075s
Epoch: 0096 loss_train: 0.8822 acc_train: 0.7786 loss_val: 1.0266 acc_val: 0.7900 time: 0.0075s
Epoch: 0097 loss_train: 0.8434 acc_train: 0.8286 loss_val: 1.0191 acc_val: 0.7967 time: 0.0070s
Epoch: 0098 loss_train: 0.8434 acc_train: 0.8214 loss_val: 1.0118 acc_val: 0.7967 time: 0.0075s
Epoch: 0099 loss_train: 0.7982 acc_train: 0.8643 loss_val: 1.0053 acc_val: 0.7967 time: 0.0075s
Epoch: 0100 loss_train: 0.8080 acc_train: 0.8286 loss_val: 0.9992 acc_val: 0.8000 time: 0.0075s
Epoch: 0101 loss_train: 0.8088 acc_train: 0.8429 loss_val: 0.9932 acc_val: 0.8033 time: 0.0080s
Epoch: 0102 loss_train: 0.7629 acc_train

Epoch: 0183 loss_train: 0.4354 acc_train: 0.9357 loss_val: 0.7238 acc_val: 0.8100 time: 0.0075s
Epoch: 0184 loss_train: 0.4581 acc_train: 0.9286 loss_val: 0.7218 acc_val: 0.8033 time: 0.0073s
Epoch: 0185 loss_train: 0.4628 acc_train: 0.9071 loss_val: 0.7194 acc_val: 0.8167 time: 0.0092s
Epoch: 0186 loss_train: 0.4442 acc_train: 0.9214 loss_val: 0.7182 acc_val: 0.8233 time: 0.0085s
Epoch: 0187 loss_train: 0.4695 acc_train: 0.9500 loss_val: 0.7173 acc_val: 0.8233 time: 0.0080s
Epoch: 0188 loss_train: 0.4952 acc_train: 0.8786 loss_val: 0.7178 acc_val: 0.8267 time: 0.0081s
Epoch: 0189 loss_train: 0.4450 acc_train: 0.9429 loss_val: 0.7190 acc_val: 0.8133 time: 0.0075s
Epoch: 0190 loss_train: 0.4299 acc_train: 0.9429 loss_val: 0.7199 acc_val: 0.8100 time: 0.0079s
Epoch: 0191 loss_train: 0.4375 acc_train: 0.9786 loss_val: 0.7195 acc_val: 0.8133 time: 0.0095s
Epoch: 0192 loss_train: 0.4396 acc_train: 0.9143 loss_val: 0.7181 acc_val: 0.8033 time: 0.0093s
Epoch: 0193 loss_train: 0.4575 acc_train

In [ ]:
# results: loss= 0.7335 accuracy= 0.8370